In [5]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from math import exp
from math import log2
from numpy.linalg import inv

In [6]:
class MyLinearUnivariateRegression:
    def __init__(self):
        self.intercept_ = 0.0
        self.coef_ = 0.0

        self.intercept2_ = 0.0
        self.coef1_ = 0.0
        self.coef2_ = 0.0

    # learn a linear univariate regression model by using training inputs (x) and outputs (y)
    def fit(self, x, y):
        sx = sum(x)
        sy = sum(y)
        sx2 = sum(i * i for i in x)
        sxy = sum(i * j for (i,j) in zip(x, y))
        w1 = (len(x) * sxy - sx * sy) / (len(x) * sx2 - sx * sx + 0.0000000000000000000000000000000000001)
        w0 = (sy - w1 * sx) / len(x)
        self.intercept_, self.coef_ =  w0, w1

    def fit2(self,x1,x2,y):
        avgy = sum(y)/len(y)
        avgx1 = sum(x1)/len(x1)
        avgx2 = sum(x2)/len(x2)

        w1 = sum(i*i for i in x2)*sum(i*j for i,j in zip(x1,y)) - sum(i*j for i,j in zip(x1,x2))*sum(i*j for i,j in zip(x2,y))
        w1 = w1 / (sum(i*i for i in x1)*sum(i*i for i in x2)*pow(sum(i*j for i,j in zip(x1,x2)),2))

        w2 = sum(i*i for i in x1)*sum(i*j for i,j in zip(x2,y)) - sum(i*j for i,j in zip(x1,x2))*sum(i*j for i,j in zip(x1,y))
        w2 = w2/ (sum(i*i for i in x1)*sum(i*i for i in x2)*pow(sum(i*j for i,j in zip(x1,x2)),2))

        w0 = avgy - w1*avgx1-w2*avgx2

        self.intercept2_ = w0
        self.coef1_ = w1
        self.coef2_ = w2


    # predict the outputs for some new inputs (by using the learnt model)
    def predict(self, x):
        if (isinstance(x[0], list)):
            return [self.intercept_ + self.coef_ * val[0] for val in x]
        else:
            return [self.intercept_ + self.coef_ * val for val in x]

    def predict2(self,x1,x2):
        if(isinstance(x1,list) and isinstance(x2,list)):
            return [self.intercept2_ + self.coef1_ * val[0] + self.coef2_ * val[1] for val in zip(x1,x2)]
        else:
            return self.intercept2_ + self.coef1_ * x1 + self.coef2_ * x2

In [7]:
def loadData(fileName, inputVariabName, outputVariabName):
    data = []
    dataNames = []
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                dataNames = row
            else:
                data.append(row)
            line_count += 1
    selectedVariable = dataNames.index(inputVariabName)
    inputs = [float(data[i][selectedVariable]) for i in range(len(data)) if data[i][selectedVariable] != '' and data[i][selectedVariable] != 0]
    selectedOutput = dataNames.index(outputVariabName)
    outputs = [float(data[i][selectedOutput]) for i in range(len(data)) if data[i][selectedVariable] != '' and data[i][selectedVariable] != 0]

    return inputs, outputs

def load2Data(fileName, firstVariabName, secondVariableName, outputVariabName):
    data = []
    dataNames = []
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                dataNames = row
            else:
                data.append(row)
            line_count += 1
    selectedOutput = dataNames.index(outputVariabName)
    selectedVariable1 = dataNames.index(firstVariabName)
    selectedVariable2 = dataNames.index(secondVariableName)

    inputs1 = []
    inputs2 = []
    outputs = []

    for i in range(len(data)):
        i1 = data[i][selectedVariable1]
        i2 = data[i][selectedVariable2]
        out = data[i][selectedOutput]

        if i1 != '' and i1 != 0 and i2 != '' and i2 != 0:
            inputs1.append(float(i1))
            inputs2.append(float(i2))
            outputs.append(float(out))

    return inputs1,inputs2,outputs


In [18]:
inputsF, outsF = loadData('data/v1_world-happiness-report-2017.csv','Family','Happiness.Score')
inputsPib, inputsFreedom, outsPF = load2Data('data/v1_world-happiness-report-2017.csv','Economy..GDP.per.Capita.','Freedom','Happiness.Score')

In [ ]:
# cu tool pentru v1

In [19]:
# Familie
def train_and_evaluate_model(inputsF, outsF, example_input):
    df = pd.DataFrame({'Family': inputsF, 'Happiness': outsF})

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        df[['Family']], df['Happiness'], test_size=0.2, random_state=42
    )

    # Train model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    # Predict example
    example_df = pd.DataFrame({'Family': example_input})
    predicted = model.predict(example_df)
    print("Predicted Happiness:", predicted)

    return model

model_family = train_and_evaluate_model(inputsF, outsF, [1.7])


Mean Squared Error: 1.2990703065242777
Predicted Happiness: [6.08618083]


In [20]:
# Pib & Libertate
def train_and_evaluate_model(features, target, example_input, feature_names):
    df = pd.DataFrame({**{name: col for name, col in zip(feature_names, features)}, 'Happiness': target})

    # Split data
    X = df[feature_names]
    y = df['Happiness']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Squared Error:", mse)

    # Predict example
    example_df = pd.DataFrame([example_input], columns=feature_names)
    predicted = model.predict(example_df)
    print("Predicted Happiness:", predicted)

    return model

# Example call
model_pib_freedom = train_and_evaluate_model(
    features=[inputsPib, inputsFreedom],
    target=outsPF,
    example_input=[1.61, 0.61],
    feature_names=['Pib', 'Freedom']
)

Mean Squared Error: 0.2303788880955436
Predicted Happiness: [6.94729923]


In [ ]:
# fara tool pentru v1

In [21]:
# Familie
def train_and_evaluate_model_univariate(inputs, outputs, example_input):
    model = MyLinearUnivariateRegression()
    model.fit(inputs, outputs)

    # Predict and evaluate
    predicted = model.predict(example_input)
    print("Predicted Happiness:", predicted)

    return model

model_family_univariate = train_and_evaluate_model_univariate(inputsF, outsF, [1.7])

Predicted Happiness: [5.528506822884081e+25]


In [22]:
# Pib & Libertate
def train_and_evaluate_model_multivariate(inputs1, inputs2, outputs, example_input):
    model = MyLinearUnivariateRegression()
    model.fit2(inputs1, inputs2, outputs)

    # Predict and evaluate
    predicted = model.predict2(example_input[0], example_input[1])
    print("Predicted Happiness:", predicted)

    return model

model_pib_freedom_univariate = train_and_evaluate_model_multivariate(inputsPib, inputsFreedom, outsPF, [1.61, 0.61])

Predicted Happiness: 5.3541293855733265
